# 🦅 AVCI - Yüksek Oran Avcısı Eğitim Paneli
Bu panel, her bir hedef oran (1.5x, 3.0x, 10.0x vb.) için **ayrı ayrı** eğitim ve optimizasyon yapmanı sağlar.
Ayrıca her eğitimin sonunda:
1.  **Güven Grafiği:** Modelin hangi oyunlara ne kadar güvendiği.
2.  **Kâr/Zarar Grafiği:** Olası kazanç simülasyonu.

---

In [ ]:
# 1. Hazırlık ve Bağlantılar
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/MyDrive/newproje/avci

!pip install optuna lightgbm streamlit matplotlib pandas

In [ ]:
# 2. Veri ve Özellikleri Yükle (Tek Seferde)
from train_avci import load_and_prep, train_target, visualize_performance

# Veriyi hazırlayıp hafızaya alalım
df = load_and_prep(limit=100000)
print(f"Veri Hazır: {len(df)} satır.")

## 🎯 Hedef 1: 1.50x (Güvenli Liman)

In [ ]:
target = 1.5
model, X_val, y_val, _ = train_target(df, target, epochs=30)
visualize_performance(model, X_val, y_val, target)

## 🎯 Hedef 2: 3.00x (Orta Risk)

In [ ]:
target = 3.0
model, X_val, y_val, _ = train_target(df, target, epochs=30)
visualize_performance(model, X_val, y_val, target)

## 🎯 Hedef 3: 5.00x (Av Başlıyor)

In [ ]:
target = 5.0
model, X_val, y_val, _ = train_target(df, target, epochs=50)
visualize_performance(model, X_val, y_val, target)

## 🎯 Hedef 4: 10.00x (Büyük Vurgun)
*Dikkat: Bu model çok nadir sinyal vermeli.*

In [ ]:
target = 10.0
model, X_val, y_val, _ = train_target(df, target, epochs=50)
visualize_performance(model, X_val, y_val, target)

## 🎯 Hedef 5: 50.00x - 100.00x (Rüya Avcısı)

In [ ]:
# Çok yüksek hedefler için epochs sayısını artırıyoruz
target = 50.0
model, X_val, y_val, _ = train_target(df, target, epochs=100)
visualize_performance(model, X_val, y_val, target)